In [50]:
# 步驟 1: 匯入必要的庫
import pandas as pd
import re
import jieba
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.sparse import hstack, csr_matrix
import math

# ----------------------------------------------------
# 步驟 2: 設定檔案路徑與名稱
file_name = 'train.csv' 
file_path = file_name 
# ----------------------------------------------------

# 步驟 3 & 4: 檢查檔案並讀取
if not os.path.exists(file_path):
    print(f"錯誤: 找不到檔案 '{file_name}'。正在使用演示數據。")
    data = {
        'name': ['! 十八麻油鴨粽-熱', '"奇妮"醫療用束帶(未滅菌)', 'NT$1490品牌男運動鞋', '"和豐國際" 洗鼻器(未滅菌)-Mycare邁康洗鼻器', '$135 拼圖'],
        'price': [50, 1020, 1490, 400, 135]
    }
    df = pd.DataFrame(data)
else:
    try:
        df = pd.read_csv(file_path)
        print(f"成功讀取檔案: '{file_name}'")
    except Exception as e:
        print(f"讀取檔案時發生錯誤: {e}")
        df = None

# 步驟 5: 顯示讀取結果的概覽
if df is not None:
    print("\n--- 數據集概覽 ---")
    print(f"數據集形狀 (列數, 行數): {df.shape}")
    print("數據集前 5 行:")
    print(df.head())
    print("\n欄位資訊:")
    df.info()
else:
    # 停止後續執行
    raise SystemExit("數據讀取失敗，程式停止。")

成功讀取檔案: 'train.csv'

--- 數據集概覽 ---
數據集形狀 (列數, 行數): (495387, 2)
數據集前 5 行:
          name  price
0   ! 十八麻油鴨粽-熱     50
1  ! 大福 小米肉粽-凍    104
2    ! 筍芋竹香粽-凍    298
3   ! 總 蛋黃肉粽-熱     42
4   ! 鮑魚干貝荷飯-凍    258

欄位資訊:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495387 entries, 0 to 495386
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    495387 non-null  object
 1   price   495387 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.6+ MB


In [51]:
# ----------------------------------------
# 0.1 從商品名稱提取初始價格特徵
# ----------------------------------------
price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

def extract_initial_price(name):
    """從商品名稱提取以 $ 或 NT 開頭的價格數值。若未找到價格，回傳 0。"""
    if pd.isna(name):
        return 0.0
    match = price_pattern.search(str(name))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return 0.0
    return 0.0

df['initial_price'] = df['name'].apply(extract_initial_price).fillna(0.0)
print("--- 初始價格特徵提取結果 ---")
print(df[['name', 'initial_price']].head())

# ----------------------------------------
# 0.2 綜合文本清洗函數 (保留貨幣符號)
# ----------------------------------------
def comprehensive_clean_name_v3(name):
    """保留貨幣符號、數字、字母、中文並移除其他符號。"""
    name = str(name).strip().lower()
    name = name.replace("&quot;", " ")
    name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

df['cleaned_name'] = df['name'].apply(comprehensive_clean_name_v3)
print("\n--- 綜合文本清洗結果 (保留貨幣符號) ---")
print(df[['name', 'cleaned_name']].head())

# ----------------------------------------
# 0.3 目標變量轉換 (Price -> Log Price)
# ----------------------------------------
df['log_price'] = np.log1p(df['price'])
print(f"\n目標變量 Log 轉換完成，新欄位為 'log_price'")


--- 初始價格特徵提取結果 ---
          name  initial_price
0   ! 十八麻油鴨粽-熱            0.0
1  ! 大福 小米肉粽-凍            0.0
2    ! 筍芋竹香粽-凍            0.0
3   ! 總 蛋黃肉粽-熱            0.0
4   ! 鮑魚干貝荷飯-凍            0.0

--- 綜合文本清洗結果 (保留貨幣符號) ---
          name cleaned_name
0   ! 十八麻油鴨粽-熱     十八麻油鴨粽 熱
1  ! 大福 小米肉粽-凍    大福 小米肉粽 凍
2    ! 筍芋竹香粽-凍      筍芋竹香粽 凍
3   ! 總 蛋黃肉粽-熱     總 蛋黃肉粽 熱
4   ! 鮑魚干貝荷飯-凍     鮑魚干貝荷飯 凍

目標變量 Log 轉換完成，新欄位為 'log_price'


In [52]:
# ----------------------------------------
# 1. 中文分詞 (Tokenization)
# ----------------------------------------
# 建議您安裝 jieba: !pip install jieba

stopwords = set()  # 可在此載入領域停用詞

def segment_chinese_text(text):
    """使用 Jieba 進行精確模式分詞，並移除停用詞。"""
    words = jieba.cut(text, cut_all=False)
    tokens = [word for word in words if word.strip() != "" and word not in stopwords]
    return " ".join(tokens)

df['name_tokens'] = df['cleaned_name'].apply(segment_chinese_text)
print("--- 中文分詞結果 ---")
print(df[['cleaned_name', 'name_tokens']].head())

# ----------------------------------------
# 2. 準備訓練數據 (同時分割文本與初始價格特徵)
# ----------------------------------------
X_train_text, X_test_text, X_train_initial_price, X_test_initial_price, y_train, y_test = train_test_split(
    df['name_tokens'],
    df['initial_price'],
    df['log_price'],
    test_size=0.2,
    random_state=42
)

print(f"數據集分割完成: 訓練集大小={X_train_text.shape[0]}, 測試集大小={X_test_text.shape[0]}")


--- 中文分詞結果 ---
  cleaned_name  name_tokens
0     十八麻油鴨粽 熱  十八 麻油 鴨 粽 熱
1    大福 小米肉粽 凍  大福 小米 肉 粽 凍
2      筍芋竹香粽 凍    筍芋 竹香 粽 凍
3     總 蛋黃肉粽 熱    總 蛋黃肉 粽 熱
4     鮑魚干貝荷飯 凍   鮑魚 干貝 荷飯 凍
數據集分割完成: 訓練集大小=396309, 測試集大小=99078


In [53]:
# ----------------------------------------
# 3. TF-IDF 向量化
# ----------------------------------------

# 按指示使用指定參數進行向量化
tfidf = TfidfVectorizer(
    max_features=50000,
    ngram_range=(1, 2)
)

# 在訓練集上 fit (學習詞彙表和權重)
X_train_vec = tfidf.fit_transform(X_train_text)

# 在測試集上 transform (使用訓練集的詞彙表)
X_test_vec = tfidf.transform(X_test_text)

print(f"\n--- TF-IDF 向量化結果 ---")
print(f"總詞彙表大小 (特徵維度): {X_train_vec.shape[1]}")
print(f"訓練集特徵矩陣形狀: {X_train_vec.shape}")
print(f"測試集特徵矩陣形狀: {X_test_vec.shape}")

# ----------------------------------------
# 3.1 數值特徵標準化與特徵拼接
# ----------------------------------------
scaler = StandardScaler()
X_train_price_scaled = scaler.fit_transform(X_train_initial_price.to_numpy().reshape(-1, 1))
X_test_price_scaled = scaler.transform(X_test_initial_price.to_numpy().reshape(-1, 1))

# 將標準化後的數值特徵轉為稀疏矩陣並與 TF-IDF 特徵拼接
X_train_final = hstack([X_train_vec, csr_matrix(X_train_price_scaled)])
X_test_final = hstack([X_test_vec, csr_matrix(X_test_price_scaled)])

print(f"--- 特徵拼接結果 ---")
print(f"訓練集最終特徵矩陣形狀: {X_train_final.shape}")
print(f"測試集最終特徵矩陣形狀: {X_test_final.shape}")

# ----------------------------------------
# 4. 模型訓練 (Ridge Regression)
# ----------------------------------------
ridge_model = Ridge(alpha=1.0)

print("--- 模型訓練 (Ridge) ---")
# 使用結合後的特徵進行訓練
ridge_model.fit(X_train_final, y_train)
print("Ridge 模型訓練完成。")

# 保留模型實例供後續使用
model = ridge_model



--- TF-IDF 向量化結果 ---
總詞彙表大小 (特徵維度): 50000
訓練集特徵矩陣形狀: (396309, 50000)
測試集特徵矩陣形狀: (99078, 50000)
--- 特徵拼接結果 ---
訓練集最終特徵矩陣形狀: (396309, 50001)
測試集最終特徵矩陣形狀: (99078, 50001)
--- 模型訓練 (Ridge) ---
Ridge 模型訓練完成。


In [54]:
# ----------------------------------------
# 5. 模型評估
# ----------------------------------------

# 使用最終特徵矩陣進行預測
y_pred_log = model.predict(X_test_final)

# 轉換回原始價格 (Price)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# 計算評估指標
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))

print(f"\n--- 模型評估 ---")
print(f"平均絕對誤差 (MAE) - 原始價格: {mae:.2f}")
print(f"均方根誤差 (RMSE) - 原始價格: {rmse:.2f}")

# 示範預測結果
results_df = pd.DataFrame({
    'Actual_Price': y_true,
    'Predicted_Price': y_pred
})
print("部分預測結果:")
print(results_df.head())



--- 模型評估 ---
平均絕對誤差 (MAE) - 原始價格: 709.30
均方根誤差 (RMSE) - 原始價格: 5342.11
部分預測結果:
        Actual_Price  Predicted_Price
412125         230.0       235.760484
207881       41310.0     34267.467955
353876        1199.0       737.647625
210982         890.0       620.221040
300354         495.0       807.706882


## Test

In [55]:
# 匯入必要的庫
import pandas as pd
import re
import jieba
import numpy as np
import os
from scipy.sparse import csr_matrix, hstack

# ----------------------------------------------------
# 1. 讀取測試集
# ----------------------------------------------------
test_file_path = 'test.csv'
try:
    df_test = pd.read_csv(test_file_path)
    print(f"成功讀取測試集: {test_file_path}")
    print(f"測試集形狀: {df_test.shape}")
except FileNotFoundError:
    print(f"錯誤: 找不到檔案 '{test_file_path}'。正在創建演示數據。")
    df_test = pd.DataFrame({
        'name': [
            '! 亞洲特級香米-熱',
            '"奇妮"最新款醫療用束帶(未滅菌) GH20',
            'NT$1590品牌女運動鞋-2025版',
            '$99 玩具積木'
        ]
    })


# ----------------------------------------
# 2. 定義清洗與特徵工程函數 (需與訓練階段一致)
# ----------------------------------------
price_pattern = re.compile(r'(?i)(?:\$|nt\$?)\s*(\d+(?:\.\d+)?)')

def extract_initial_price(name):
    if pd.isna(name):
        return 0.0
    match = price_pattern.search(str(name))
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return 0.0
    return 0.0

def comprehensive_clean_name_v3(name):
    name = str(name).strip().lower()
    name = name.replace("&quot;", " ")
    name = re.sub(r'[^$\u4e00-\u9fa5a-z0-9\s]', ' ', name)
    name = re.sub(r"\s+", " ", name).strip()
    return name

stopwords = set()

def segment_chinese_text(text):
    words = jieba.cut(text, cut_all=False)
    tokens = [word for word in words if word.strip() != "" and word not in stopwords]
    return " ".join(tokens)


成功讀取測試集: test.csv
測試集形狀: (55043, 1)


In [56]:
# ----------------------------------------
# 1. 在測試集上套用清洗、分詞與初始價格提取
# ----------------------------------------
df_test['initial_price'] = df_test['name'].apply(extract_initial_price).fillna(0.0)
df_test['cleaned_name'] = df_test['name'].apply(comprehensive_clean_name_v3)
df_test['name_tokens'] = df_test['cleaned_name'].apply(segment_chinese_text)

print("\n--- 測試集清洗與特徵工程結果 (前 5 筆) ---")
print(df_test[['name', 'initial_price', 'name_tokens']].head())


# ----------------------------------------
# 2. 應用 TF-IDF 轉換與數值特徵標準化
# ----------------------------------------
if 'tfidf' not in locals():
    raise NameError("錯誤: 找不到 'tfidf' 變數。請先運行訓練集的程式碼。")
if 'scaler' not in locals():
    raise NameError("錯誤: 找不到 'scaler' 變數。請先運行訓練集的程式碼。")

X_test_vec_final = tfidf.transform(df_test['name_tokens'])
initial_price_scaled = scaler.transform(df_test["initial_price"].to_numpy().reshape(-1, 1))
X_test_final = hstack([X_test_vec_final, csr_matrix(initial_price_scaled)])

print(f"\n--- 測試集特徵矩陣 ---")
print(f"TF-IDF 特徵矩陣形狀: {X_test_vec_final.shape}")
print(f"最終特徵矩陣形狀: {X_test_final.shape}")


# ----------------------------------------
# 3. 模型預測 (使用已訓練好的 Ridge 模型)
# ----------------------------------------
if 'model' not in locals():
    raise NameError("錯誤: 找不到 'model' 變數。請先運行訓練集的程式碼。")

# 預測 log_price 並還原至原始價格
y_pred_log_test = model.predict(X_test_final)
y_pred_price_test = np.expm1(y_pred_log_test)
df_test['price'] = y_pred_price_test



--- 測試集清洗與特徵工程結果 (前 5 筆) ---
                          name  initial_price                   name_tokens
0                    !(呷)蛋黃粽-熱            0.0                      呷 蛋黃 粽 熱
1  #13-PILOT可擦印章 FRIXION stamp            0.0  13 pilot 可擦 印章 frixion stamp
2  #17-PILOT可擦印章 FRIXION stamp            0.0  17 pilot 可擦 印章 frixion stamp
3                #304隔熱湯碗組16CM            0.0              304 隔熱 湯碗 組 16cm
4               #316方形筷盒組-20cm            0.0              316 方形 筷盒 組 20cm

--- 測試集特徵矩陣 ---
TF-IDF 特徵矩陣形狀: (55043, 50000)
最終特徵矩陣形狀: (55043, 50001)


In [57]:
# ----------------------------------------
# 4. 輸出結果
# ----------------------------------------

print("\n--- 最終預測結果 (前 5 筆) ---")
print(df_test[['name', 'price']].head())

# 儲存結果到 CSV 文件
output_filename = 'test_predictions.csv'
df_test[['name', 'price']].to_csv(output_filename, index=False, float_format='%.2f')

print(f"\n預測結果已儲存到 '{output_filename}'")


--- 最終預測結果 (前 5 筆) ---
                          name       price
0                    !(呷)蛋黃粽-熱  174.916932
1  #13-PILOT可擦印章 FRIXION stamp  102.425269
2  #17-PILOT可擦印章 FRIXION stamp   91.428596
3                #304隔熱湯碗組16CM  166.532301
4               #316方形筷盒組-20cm  173.735604

預測結果已儲存到 'test_predictions.csv'
